In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
import os
os.chdir('/gdrive/My Drive/NLP_Project_Programs/Retrofitting_ESA')
os.listdir()

['Graphs',
 'Datasets',
 'OtherFiles',
 'Evaluation_Benchmarks',
 'TOEFL Task.ipynb',
 'Wikipedia-RG65_10titles.xml',
 'Wikipedia-RG65_20titles.xml',
 'Wikipedia-MEN_5titles.xml',
 'Wikipedia-MEN_3titles.xml',
 'Eval_Dataset_String.ipynb',
 'Sentiment_Analysis.ipynb',
 'titles_extract.ipynb',
 'explicit_semantic_anlaysis+retrofitting.ipynb',
 'Word Similarity Tasks.ipynb',
 'Graph_Processing.ipynb']

In [0]:
import pandas as pd
import numpy as np
from collections import defaultdict
import json
from tqdm.notebook import tqdm
import random
random.seed(10)

In [0]:
print(random.random())

0.4288890546751146


In [0]:
df = pd.read_csv('./Graphs/enwiki.wikilink_graph.2018-03-01.csv.gz', compression='gzip', header=0, sep='\t')

In [0]:
df.head()

In [0]:
def get_wiki_corpus_titles():
  count = 0
  corpus_titles = []
  # Find all text files within temp folder.
  # breakup = False
  for filename in tqdm(os.listdir('./Datasets/ws6/temp/')):
      if filename.endswith(".raw"):
          # print('-'*25, filename)
          file = os.path.join('./Datasets/ws6/temp/', filename)
          with open(file) as json_file:
              concepts = json.load(json_file)
              for concept in concepts.keys():
                  if concepts[concept]['links_out']:
                    len_document = len(concepts[concept]['text'].split(" "))
                    if len_document>0:
                      # if len_document>200 and len_document<250:
                      #   if random.random()<0.2: # 10% of the docs
                      # print(concepts[concept]['text'].split(" "), concept)
                      # print(len(concepts[concept]['text'].split(" ")))
                      corpus_titles.append(concept)
                      count+=1
                      # break
                      # print(concept)
                      # print(concepts[concept]['links_out'])
                      # breakup =True
      else:
        continue
      # break
      # if breakup:
      #   break
  return corpus_titles

corpus_titles = get_wiki_corpus_titles()
corpus_titles_set = set(corpus_titles)
len(corpus_titles_set), len(corpus_titles)

(2555, 2555)

In [0]:
len(corpus_titles_set), len(corpus_titles)

(2555, 2555)

In [0]:
def generate_graph_1():
  """
  Using Pre-generated Graph reduce the Graph and save in the required format for retrofitting.
  Note: Articles are separated by '--' instead of ' ' because there could be multiple words in the title.
  """
  titles_from = df['page_title_from'].values
  titles_to = df['page_title_to'].values
  # titles_from_set = set(titles_from)
  # titles_to_set = set(titles_to)
  # graph_titles_set = titles_from.union(titles_to)

  reduced_edges = []
  for x, y in tqdm(zip(titles_from, titles_to)):
    if (x in corpus_titles_set) and (y in corpus_titles_set):
      reduced_edges.append((x,y))

  wiki_graph = defaultdict(list)
  for edge in tqdm(reduced_edges):
    wiki_graph[edge[0]].append(edge[1])
    wiki_graph[edge[1]].append(edge[0])

  wiki_graph_set = {}
  for key, value in tqdm(wiki_graph.items()):
    wiki_graph_set[key] = set(value)

  file = open("wiki_graph_1.txt","w") 
  for key, value in tqdm(wiki_graph_set.items()):
    file.write(key + "--" + "--".join(list(value)) + "\n")
  file.close()
  return wiki_graph_set
wiki_graph_set = generate_graph_1()

In [0]:
def generate_graph_2():
    """
    Generate Graph using 'links_out' in Wikipedia Dump and save in the required format for retrofitting.
    Note: Articles are separated by '--' instead of ' ' because there could be multiple words in the title.
    """
  wiki_graph_set = {}
  for filename in tqdm(os.listdir('./Datasets/temp/')):
      if filename.endswith(".raw"):
          # print('-'*25, filename)
          file = os.path.join('./Datasets/temp/', filename)
          with open(file) as json_file:
              concepts = json.load(json_file)
              for concept in concepts.keys():
                  if concepts[concept]['links_out']:
                    wiki_graph_set[concept] = set(concepts[concept]['links_out'])
      else:
        continue
  file = open("Graphs/wiki_graph_2.txt","w")
  for key, value in tqdm(wiki_graph_set.items()):
    file.write(key + "--" + "--".join(list(value)) + "\n")
  file.close()
  return wiki_graph_set
wiki_graph_set = generate_graph_2()

In [0]:
def generate_graph_3():
  """
  Generate Graph using 'links_out' in Wikipedia Dump and save in the required format for retrofitting.
  Only this time limit to those articles with greater than 1000 tokens/words.
  Note: Articles are separated by '--' instead of ' ' because there could be multiple words in the title.
  """
  wiki_graph_set = defaultdict(list)
  for filename in tqdm(os.listdir('./Datasets/ws6/temp/')):
      if filename.endswith(".raw"):
          # print('-'*25, filename)
          file = os.path.join('./Datasets/ws6/temp/', filename)
          with open(file) as json_file:
              concepts = json.load(json_file)
              for concept in concepts.keys():
                if concept in corpus_titles_set:
                  if concepts[concept]['links_out']:
                    len_document = len(concepts[concept]['text'].split(" "))
                    if len_document>0:
                      neighbour_concepts = set(concepts[concept]['links_out'])
                      for neighbour_concept in neighbour_concepts:
                        if neighbour_concept in corpus_titles_set:
                          wiki_graph_set[concept].append(neighbour_concept)
      else:
        continue
  file = open("Graphs/wiki_graph_3_ws6.txt","w")
  for key, value in tqdm(wiki_graph_set.items()):
    file.write(key + "--" + "--".join(list(value)) + "\n")
  file.close()
  return wiki_graph_set
wiki_graph_set = generate_graph_3()

In [0]:
len(wiki_graph_set)

1768

In [0]:
''' Read the graph with Wiki article relations as a dictionary '''
def read_lexicon(filename):
  lexicon = {}
  for line in open(filename, 'r',encoding = "utf8"):
    words = line.strip().split("--")
    lexicon[words[0]] = [word for word in words[1:]]
  return lexicon

lexicon = read_lexicon(filename = "./Graphs/wiki_graph_2.txt")

In [4]:
''' Read the graph with Wiki article relations as a dictionary '''
def find_nodes_edges(filename):
  lexicon = {}
  n_nodes = 0
  edges = []
  for line in open(filename, 'r',encoding = "utf8"):
    n_nodes+=1
    words = line.strip().split("--")
    concept = words[0]
    for word in words[1:]:
      edge = "--".join(sorted([concept, word]))
      edges.append(edge)
  n_edges = len(set(edges))
  print(filename, '\nEdges:', n_edges, '\tnodes:', n_nodes)
  return lexicon

graphs = [
          "./Graphs/wiki_graph_2_v4.txt", 
          "./Graphs/wiki_graph_2_rg.txt",
          "./Graphs/wiki_graph_2_rg20.txt", 
          "./Graphs/wiki_graph_2_men3.txt", 
          "./Graphs/wiki_graph_2_men5.txt",
          "./Graphs/wiki_graph_3_ws6.txt"
          ]

for graph in graphs:
  lexicon = find_nodes_edges(filename = graph)

./Graphs/wiki_graph_2_v4.txt 
Edges: 4637 	nodes: 1459
./Graphs/wiki_graph_2_rg.txt 
Edges: 179 	nodes: 150
./Graphs/wiki_graph_2_rg20.txt 
Edges: 445 	nodes: 338
./Graphs/wiki_graph_2_men3.txt 
Edges: 8225 	nodes: 1775
./Graphs/wiki_graph_2_men5.txt 
Edges: 13244 	nodes: 3026
./Graphs/wiki_graph_3_ws6.txt 
Edges: 5626 	nodes: 1768


In [0]:
"--".join(sorted(['Apple', 'Banana']))

'Apple--Banana'

In [0]:
lexicon['Artificial intelligence']

['Technological Singularity',
 'Brad Rutter',
 'Neural Network',
 'Natural Language Processing',
 'Linguistics',
 'John Mccarthy',
 'Hephaestus',
 'Management',
 'Chess',
 'Problem Solving',
 'Perception',
 'Crete',
 'Fuzzy Logic',
 'Learning',
 'Department Of Defense',
 'Lisp Machine',
 'Data Mining',
 'Ken Jennings',
 'Expert System',
 'Jeopardy!',
 'Humanoid',
 'Alan Turing',
 'Emotion',
 'Information Theory',
 'Cognition',
 'Optical Character Recognition',
 'Philosophy',
 'Theory Of Computation',
 'Machine',
 'Machine Learning',
 'Computer',
 'Andreas Kaplan',
 'R.U.R.',
 'Talos',
 'Cybernetics',
 'Common Sense',
 'Frankenstein',
 'Psychology',
 'Neuroscience',
 'Computer Science',
 'Mathematics',
 'Self-Driving Cars',
 'Hero Of Alexandria',
 'Decision Making',
 'Bayesian Network',
 'Emotional Intelligence',
 'Dartmouth College',
 'Garry Kasparov',
 'Lee Sedol',
 'Computer Program',
 'Al-Jazari',
 'Neurology',
 'Ai Winter',
 'Alphago',
 'Watson (Computer)',
 'Go',
 'Brainstorming',

In [0]:
for index, (key, value) in enumerate(lexicon.items()):
  print(key, '\n', value, '\n')
  if index>10: break

Elephant 
 ['Vein', 'Angola', 'Acacia'] 

String theory 
 ['Antiparticle'] 

Saturn 
 ['Silicon', 'Rotation'] 

Uranus 
 ['John Flamsteed'] 

Turkey 
 ['Achaemenid Empire'] 

Norway 
 ['Map'] 

Metal 
 ['Mining'] 

Australia 
 ['Bob Hawke', 'Queensland', 'Mining'] 

Earthquake 
 ['United States Geological Survey'] 

Denmark 
 ['Conservatism'] 

Christmas 
 ['Messiah'] 

Atom 
 ['Xenon'] 



In [0]:
import json
from tqdm.notebook import tqdm

# count = 0
corpus_titles = []
docs_length = []
# Find all text files within temp folder.
breakup = False
for filename in tqdm(os.listdir('./Datasets/ws/temp/')):
    if filename.endswith(".raw"):
        # print('-'*25, filename)
        file = os.path.join('./Datasets/ws/temp/', filename)
        with open(file) as json_file:
            concepts = json.load(json_file)
            for concept in concepts.keys():
                if concepts[concept]['links_out']:

                  docs_length.append(len(concepts[concept]['text'].split(" ")))
                  corpus_titles.append(concept)
                #   count+=1
                # if concept == "Anarchism":
                  # print(concepts[concept])
                  # print(concepts[concept]['text'])
                  # print(concepts[concept]['links_out'])
                  # breakup =True
    else:
      continue
    # print(docs_length, corpus_titles)
    # break
    # if breakup:
    #   break

lengths = [i*100 for i in range(1, 16)]
for length in lengths:
  print("Length :", length, "<=> Num Docs :", np.sum(np.array(docs_length)>length))


Length : 100 <=> Num Docs : 53980
Length : 200 <=> Num Docs : 24894
Length : 300 <=> Num Docs : 15236
Length : 400 <=> Num Docs : 10357
Length : 500 <=> Num Docs : 7530
Length : 600 <=> Num Docs : 5714
Length : 700 <=> Num Docs : 4532
Length : 800 <=> Num Docs : 3637
Length : 900 <=> Num Docs : 3013
Length : 1000 <=> Num Docs : 2521
Length : 1100 <=> Num Docs : 2101
Length : 1200 <=> Num Docs : 1838
Length : 1300 <=> Num Docs : 1619
Length : 1400 <=> Num Docs : 1433
Length : 1500 <=> Num Docs : 1251


In [0]:
import json
from tqdm.notebook import tqdm

# count = 0
corpus_titles = []
docs_length = []
# Find all text files within temp folder.
breakup = False
for filename in tqdm(os.listdir('./Datasets/ws/temp/')):
    if filename.endswith(".raw"):
        # print('-'*25, filename)
        file = os.path.join('./Datasets/ws/temp/', filename)
        with open(file) as json_file:
            concepts = json.load(json_file)
            for concept in concepts.keys():
                if concepts[concept]['links_out']:

                  docs_length.append(len(concepts[concept]['text'].split(" ")))
                  corpus_titles.append(concept)
                #   count+=1
                # if concept == "Anarchism":
                  # print(concepts[concept])
                  # print(concepts[concept]['text'])
                  # print(concepts[concept]['links_out'])
                  # breakup =True
    else:
      continue
    # print(docs_length, corpus_titles)
    # break
    # if breakup:
    #   break

lengths = [i*100 for i in range(0, 16)]
for length in lengths:
  print("Length :", length, "<=> Num Docs :", np.sum(np.array(docs_length)>length))


Length : 0 <=> Num Docs : 2132
Length : 100 <=> Num Docs : 1836
Length : 200 <=> Num Docs : 1739
Length : 300 <=> Num Docs : 1652
Length : 400 <=> Num Docs : 1559
Length : 500 <=> Num Docs : 1487
Length : 600 <=> Num Docs : 1413
Length : 700 <=> Num Docs : 1339
Length : 800 <=> Num Docs : 1289
Length : 900 <=> Num Docs : 1251
Length : 1000 <=> Num Docs : 1205
Length : 1100 <=> Num Docs : 1163
Length : 1200 <=> Num Docs : 1128
Length : 1300 <=> Num Docs : 1092
Length : 1400 <=> Num Docs : 1054
Length : 1500 <=> Num Docs : 1020


In [0]:
'Breaza, Suceava--Romania-Commune In Romania--Suceava---County'.split('--')

['Breaza, Suceava', 'Romania-Commune In Romania', 'Suceava', '-County']

In [0]:
for key, value in tqdm(wiki_graph_set.items()):
  print(key,'\n\n', value,'\n\n',len(value))
  break

Anarchism 

 {'Simple living', 'Chicago', 'Noam Chomsky', 'Encarta', 'Limited government', 'Means of production', 'Joseph Stalin', 'Capitalism', 'Human geography', "Danish People's Party", 'Murray Bookchin', 'Black metal', 'Björk', 'V for Vendetta', 'Nonviolence', 'History of Russia', 'Esperanto', 'Karlsruhe', 'The New York Times', 'Siné', 'Bohemianism', 'Mark Rothko', 'Oxford University Press', 'June 2', 'Caliphate', 'Salvador Dalí', 'Christian terrorism', '1960s', 'Left-wing fascism', 'Lev Chernyi', 'LGBT community', 'Naturism', 'Democracy', 'Woodcut', 'Jean-Jacques Rousseau', 'Ochlocracy', 'Friedrich Engels', 'Oscar Wilde', 'Philip K. Dick', 'Homosexuality', 'Objectivism (Ayn Rand)', 'October Revolution', 'Institution', 'Socialism', 'Alexander Berkman', 'Hurricane Harvey', 'Carrara', 'Paul Virilio', 'Anarcho-syndicalism', 'Oxymoron', 'James McAuley', 'Ecclesiastical polity', 'The Day Before the Revolution', 'Natural law', 'Louise Michel', 'Anti-nuclear movement', 'September 6', 'Eri

In [0]:
graph_titles_list = list(graph_titles)
for index,title in tqdm(enumerate(graph_titles_list)):
  if title == 'Category:Charlotte, North Carolina':
    print(index)

In [0]:
len(titles_from), len(titles_to), len(graph_titles)

In [0]:
np.save('./Graphs/col2', titles_from)

In [0]:
temp = np.load('./Graphs/col2.npy', allow_pickle=True)

In [0]:
np.shape(temp)

In [0]:
unique = set(temp)

In [0]:
tfidf_matrix = np.zeros((200000, 2500))

In [0]:
import numpy as np
tfidf_matrix = np.zeros((691657, 175899))

In [0]:
from scipy.sparse import csc_matrix, lil_matrix
import numpy as np

tfidf_matrix = csc_matrix((691657, 175899), dtype=np.float32)
tfidf_matrix[10000,100] = 10000.534234


/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [0]:
tfidf_matrix.data

array([10000.534], dtype=float32)

In [0]:
tfidf_matrix = lil_matrix((691657, 175899), dtype=np.float32)
tfidf_matrix[10000,100] = 10000.534234

In [0]:
x = tfidf_matrix[:,100].toarray()

In [0]:
for temp in x:
  print(temp)

In [0]:
tfidf_matrix[:,1] = np.ones(691657)

ValueError: ignored

In [0]:
np.ones(691657)

array([1., 1., 1., ..., 1., 1., 1.])

In [0]:
import pickle

with open('Datasets/id2concept.pickle', 'rb') as handle:
    id2concept = pickle.load(handle)

FileNotFoundError: ignored

In [0]:
Took => 9.453941822052002 s(seconds)Took => 9.453941822052002 s(seconds)Took => 9.453941822052002 s(seconds)